## Демонстрация иерархического классификатора с использованием CatBoost.

Импортируем необходимые библиотеки и модули, в том числе, модули мпровизированной HierarhicalLibrary, в которых содержатся необходимые для работы иерархического классификатора классы.

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle
import tqdm

from HierarhicalLibrary import Classifier, CategoryTree, TextProcessor
from HierarhicalLibrary.Encoders import LdaEncoder, NavecEncoder, FasttextEncoder, BertEncoder

In [2]:
SEED = 1

# Method for increasing the weight of the first words of title
def word_pyramid(string: str, min_n_words: int, max_n_words: int) -> list:
    result = []
    split = string.split(' ')
    for i in range(min_n_words, max_n_words+1):
        result += split[:i]
    return ' '.join(result)

def prepare_data(full_train_data: pd.DataFrame, seed: int, valid_size: int):
    data_full = full_train_data.sample(frac=1, random_state=seed).copy()
    data_full.drop(['rating', 'feedback_quantity'], axis=1, inplace=True)
    data_full.title = data_full.title.astype('string')
    data_full.short_description = data_full.short_description.astype('string')
    data_full.fillna(value='', inplace=True)
    data_full.name_value_characteristics = data_full.name_value_characteristics.astype('string')
    data_full = data_full.assign(Document=[str(x) + ' ' + str(y) + ' ' + str(z) + ' ' + word_pyramid(x, 2, 3) for x, y, z in zip(data_full['title'], data_full['short_description'], data_full['name_value_characteristics'])])
    data_full.drop(['title', 'short_description', 'name_value_characteristics'], axis=1, inplace=True)
    data_full.Document = data_full.Document.astype('string')

    data = data_full[:-valid_size].reset_index(drop=True)
    data_valid = data_full[-valid_size:].reset_index(drop=True)
    return data, data_valid

def set_seeds(seed: int):  
    np.random.seed(seed)
    

Загружаем данные

In [3]:
cat_tree_df = pd.read_csv('categories_tree.csv', index_col=0)
full_train_data = pd.read_parquet('train.parquet')

Подготавливаем полный, тренировочный и валидационный датасеты:
перемешиваем данные в фрейме,
удаляем колонки рейтинга и кол-ва отзывов,
корректируем типы данных колонок,
заполняем пропущенные значения,
текст из колонок 'title', 'short_description' и 'name_value_characteristics' объединяем в колонку "Document", добавляем первые слова из колонки 'title', чтобы увеличить их вес.

In [4]:
set_seeds(SEED)
data, data_valid = prepare_data(full_train_data, seed=SEED, valid_size=4000)
data = data[:50000]

Для ускорения расчетов, оставим только 50000 записей, иначе, считать будет долго.

In [5]:
data

,id,category_id,Document
0,1181186,12350,Маска Masil для объёма волос 8ml /Корейская ко...
1,304936,12917,Силиконовый дорожный контейнер футляр чехол дл...
2,816714,14125,"Тканевая маска для лица с муцином улитки, 100%..."
3,1437391,11574,Браслеты из бисера Браслеты из бисера. Брасле...
4,1234938,12761,Бальзам HAUTE COUTURE LUXURY BLOND для блондир...
...,...,...,...
49995,1291099,12488,"Комплект постельного белья Считалочка, 1.5 сп,..."
49996,992089,13816,Патчи гля глаз кружевные LOVE Beauty Fox с му...
49997,529715,13613,"Пресс для чеснока MODERNO, прорезиненная ручка..."
49998,750317,12228,"Косметичка полиэстер/ПВХ розовая 19,5*11,5*11,..."


### Text Processor

Инициализируем объект энкодера (это класс, который управляет расчетами векторов скрытых представлений текстов, "эмбеддингов")

In [6]:
text_processor = TextProcessor(
    add_stop_words=[',', '.', '', '|', ':', '"', '/', ')', '(', 'a', 'х', '(:', '):', ':(', ':)', 'и']
)

Следующий код читает документы из датафрейма, выполняет токенизацию и лемматизацию средствами пакета natasha, затем, сохраняет леммы в собственную переменную Encoder.texts. Лемматизация выполняется достаточно долго, поэтому сохраняем данные на диск:

In [7]:
text_processor.lemmatize_data(data, document_col='Document', id_col='id')
text_processor.save_lemms_data('50000_set_lemm', directory='Hierarhical_with_catboost')

Lemmatize: 100%|██████████| 50000/50000 [08:04<00:00, 103.22it/s]


Загружаем леммы с диска:

In [8]:
text_processor.load_lemms_data('50000_set_lemm', directory='Hierarhical_with_catboost')

#### Энкодер на базе модели LDA gensim

In [9]:
lda_encoder = LdaEncoder()

Выполняем тренировку LDA модели gensim (скажем, на 16 тем, чтобы побыстрее работало) и сразу сохраняем на диск, модель тренируется долго:

In [10]:
lda_encoder.fit(texts=text_processor.texts, num_topics=32, passes=5, iterations=2)
lda_encoder.save_model('50000_set_model_32', directory='Hierarhical_with_catboost')

Загружаем модель с диска:

In [11]:
lda_encoder.load_model('50000_set_model_32', directory='Hierarhical_with_catboost')

Размерность эмбеддинга lda:

In [12]:
lda_encoder.transform([['foo']]).shape[1]

32

#### Энкодер на базе модели navec
параметр экспоненциального взвешивания эмбеддингов alpha=0.2

In [13]:
nevec_encoder = NavecEncoder(alpha=0.2, dim=32)

Загружаем обученную модель navec (скачана из родного репозитория).

In [14]:
nevec_encoder.load_model('navec_hudlit_v1_12B_500K_300d_100q.tar')

В случае необходимости, считаем и сохраняем матрицу снижения размерности эмбеддингов word2vec (например, на 128 векторов).

In [15]:
nevec_encoder.calc_pca(texts=text_processor.texts, sample_size=10000)
nevec_encoder.save_pca('PCA_navec.pickle', directory='Hierarhical_with_catboost')

Загружаем матрицу для понижения размерности word2vec эмбеддингов (понижение размерности выполнено для увеличения производительности, если есть желание отключить понижение размерности - можно просто не указывать параметр dim или присвоить ему значение nevec_encoder.dim=None)

In [16]:
nevec_encoder.load_pca('PCA_navec.pickle', directory='Hierarhical_with_catboost')

Размерность эмбеддинга navec:

In [17]:
nevec_encoder.transform([['foo']]).shape[1]

32

#### Расчёт составных эмбеддингов документов

Используя встроенный метод энкодера, формируем словарь эмбеддингов товаров вида {good_id(int) : embedding(np.array)}. Передаем интересующие нас функции - энкодеры LDA и Word2vec. Параметр экспоненциального взвешивания эмбеддингов word2vec, alpha=0.25.

In [18]:
encoders=[lda_encoder, nevec_encoder]

In [19]:
embeddings_dict = text_processor.make_embeddings_dict(encoders=encoders)

Сохраняем словарь эмбеддингов при необходимости - загружаем сохранённый:

In [20]:
path = os.path.join(Path(".").parent, 'Hierarhical_with_catboost', '50000_set_embs_dict.pickle')
with open(path, 'wb') as f:
    pickle.dump(embeddings_dict, f)

In [21]:
path = os.path.join(Path(".").parent, 'Hierarhical_with_catboost', '50000_set_embs_dict.pickle')
with open(path, 'rb') as f:
    embeddings_dict = pickle.load(f)

### Дерево каталога

Инициализируем дерево каталога - CategoryTree() - это класс, который хранит все узлы, необходимую информацию для обучения, а также реализует алгоритмы заполнения дерева, обхода при инференсе для определения категории товара. 
Добавляем узлы из таблицы categories_tree.csv, затем, добавляем товары из тренировочной выборки.

In [22]:
cat_tree = CategoryTree()
cat_tree.add_nodes_from_df(cat_tree_df, parent_id_col='parent_id', title_col='title')
cat_tree.add_goods_from_df(data, category_id_col='category_id', good_id_col='id')

Записываем эмбеддинги в дерево каталогов (производится расчет эмбеддингов узлов как усреднённых эмбеддингов документов, попавших в каждый узел):

In [23]:
cat_tree.update_embeddings(embeddings_dict)

Примешиваем к эмбеддингам узлов эмбеддинги их собственных описаний.

In [24]:
cat_tree.mix_in_description_embs(lambda titles: text_processor.get_embeddings(titles, encoders=encoders), weight=10)

### Классификатор

Инициализируем объект классификатора - он управляет процессом получения вероятностей принадлежности товара к узлу (predict_proba). после этого, формируем массив-датасет для тренировки глобального классификатора, сохраняем его на диск (так как памяти массив занимает очень много, можно удалить его из оперативной памяти, потом снова загрузить с диска при необходимости).

In [25]:
classifier = Classifier(catboost_parameters={'loss_function': 'Logloss',
                                    'iterations': 30,
                                    'depth': 9,
                                    'rsm': 1.0,
                                    'random_seed': 1,
                                    'learning_rate': 0.7
                                    }, tol=0.03, max_iter=500)

In [26]:
classifier.calc_global_train_array(embeddings_dict, cat_tree)
classifier.save_global_train_array('50000_set_arr', directory='Hierarhical_with_catboost')

In [27]:
classifier.load_global_train_array('50000_set_arr', directory='Hierarhical_with_catboost')

На сформированном датасете обучаем глобальный классификатор и сохраняем его. Массив больше не нужен.

In [28]:
classifier.fit_global_classifier()
classifier.save_global_classifier('50000_set_CatBoost.cbm', directory='Hierarhical_with_catboost')
classifier.delete_global_train_array()

In [29]:
classifier.load_global_classifier('50000_set_CatBoost.cbm', directory='Hierarhical_with_catboost')

Обучаем локальные веса (модель логистической регрессии в каждом из узлов дерева). Сохраняем дерево (так как считается очень долго). При необходимости - загружаем.

In [30]:
cat_tree.fit_local_weights(classifier, embeddings_dict, C=0.05, reg_count_power=0.5, verbose=False)

100%|██████████| 3370/3370 [1:22:01<00:00,  1.46s/it]  


Сохраняем, и при необходимости, загружаем дерево с рассчитанными весами.

In [31]:
cat_tree.save_tree('50000_set_tree.pickle', directory='Hierarhical_with_catboost')

In [32]:
cat_tree.load_tree('50000_set_tree.pickle', directory='Hierarhical_with_catboost')

### Тестирование модели

#### Тестирование на трейне

Формируем массив эмбеддингов для тестирования

In [33]:
begin_example = 0
end_example = 3000
train_documents = data.Document.tolist()[begin_example:end_example]
train_target = data.category_id.tolist()[begin_example:end_example]
embs = text_processor.get_embeddings(train_documents, encoders=encoders)

Выполняем поиск категорий по каталогу для каждого тестового примера

In [34]:
pred_leafs = []
for i in tqdm.tqdm(range(len(embs)), total=len(embs)):
    pred_leafs.append(cat_tree.choose_leaf(classifier = classifier, good_embedding=embs[i]))

100%|██████████| 3000/3000 [25:28<00:00,  1.96it/s]


In [35]:
print(f'Train set hF1={cat_tree.hF1_score(train_target, pred_leafs):.3f}') 

Train set hF1=0.749


#### Тестирование на отложенной выборке

Формируем массив эмбеддингов для тестирования

In [36]:
begin_example = 0
end_example = 4000
valid_documents = data_valid.Document.tolist()[begin_example:end_example]
valid_target = data_valid.category_id.tolist()[begin_example:end_example]
embs_valid = text_processor.get_embeddings(valid_documents, encoders=encoders)

Выполняем поиск категорий по каталогу для каждого тестового примера

In [37]:
pred_leafs_valid = []
for i in tqdm.tqdm(range(len(embs_valid)), total=len(embs_valid)):
    pred_leafs_valid.append(cat_tree.choose_leaf(classifier = classifier, good_embedding=embs_valid[i]))

100%|██████████| 4000/4000 [12:47<00:00,  5.21it/s]


In [38]:
print(f'Validation hF1={cat_tree.hF1_score(valid_target, pred_leafs_valid):.3f}') 

Validation hF1=0.740


В этом ноутбуке гиперпараметры и размер выборки выбраны такими, чтобы расчёты выполнялись относительно быстро. С хорошими гиперпараметрами, на полном размере выборки, удалось получить hF1=0.85, что значительно ниже бейзлайна. Кроме того, использование CatBoost как глобального классификатора, не даёт прироста качества.